# Depth Estimator Model

### Dependencies

In [44]:
import cv2
from PIL import Image
import numpy as np
import tensorflow as tf
import xgboost as xg
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

### Loading the Dataset 

In [ ]:
ds = load_dataset("0jl/NYUv2", trust_remote_code=True) 
image_paths = ds['train']['image']
depths = ds['train']['depth']

### Extracting features

In [ ]:
resnet = ResNet50(include_top=False,weights="imagenet",pooling="avg",input_shape=(224,224,3))

In [ ]:
def preprocessing(img_path):
    img = Image.open(img_path)
    img = preprocess_input(img_path)
    features = resnet.predict(np.expand_dims(img,axis=0))
    return features.flatten()

In [ ]:
type(image_paths)

list

In [ ]:
features_matrix = []
for path in image_paths:
    img = path.resize((224,224))
    img = np.array(img)
    img = np.expand_dims(img,axis = 0)
    img = img.astype("float32")
    img = preprocess_input(img)
    features_matrix.append(
        resnet.predict(img).flatten()
    )

1/1 [==============================] - 0s 14ms/step
